<a href="https://colab.research.google.com/github/rksab/NLP/blob/main/Hugging_face_Course_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The machine learning landscape is constantly evolving—but rarely has a shift felt as sweeping as the rise of large language models. LLMs haven’t just advanced the field; they’ve dominated it, capturing the imagination of both researchers and the general public in unprecedented ways.

Traditionally, NLP models were narrow in scope—trained from scratch for specific tasks like sentiment analysis, translation, or named entity recognition. Each task required its own dataset, its own architecture, its own evaluation pipeline.

That changed with the introduction of transformer architectures and large-scale pretraining. Together, they ushered in the era of LLMs.

Instead of building task-specific models, researchers began training enormous models—sometimes with hundreds of billions of parameters—on massive corpora of text. These models learned general-purpose language understanding, which could later be fine-tuned or prompted to perform a wide variety of downstream tasks.

It wasn’t just an architectural shift—it was a shift in mindset.

But do LLMs actually understand language? Not in the way humans do. They work on statistical patterns and are prone to hallucinations and bias. They require significant computational resources and text needs to be processed in a way that enables a model to learn from it.

Let's start with some hugging face magic. We'll start with pipeline, as it assumes minimal knowledge to begin with.

In [ ]:
from transformers import pipeline


In [ ]:
classifier = pipeline("sentiment-analysis")
classifier("I love to hate this course")

Since we didn't supply a model, it picked up distilbert/distilbert-base-uncased-finetuned-sst-2-english. Under the hood, it tokenized the sentence, passed the tokens to the model and assigned it a label and score.

In [ ]:
translator = pipeline("text2text-generation", model="google/flan-t5-base")
translator("Translate English to German: I love apples.")

Let's say we want to perform a classification task without training aka zero-shot classification task.

In [ ]:
clf = pipeline("zero-shot-classification")
clf("This movie was so inspiring!", candidate_labels=["sports", "politics", "entertainment"])


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="HuggingFaceTB/SmolLM2-360M")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

In [ ]:
fill = pipeline("fill-mask")
fill("The capital of France is <mask>.", top_k = 2)

Let's dig into what's going on and has been abstracted away by a simple pipeline function. First, a bit about models. There are three types of architectural variants of Transformer models: encoder, decoder and encode-decoder models.

Encoder Models: also called cuto encoding models. Attention layer gives access to the whole input, are useful when global understanding is needed. Typically, they try predicted a masked work and have access to words that come before and after it. e.g, bert-base-uncased, roberta-base. Useful for text-classification, sentence similarity, token classification etc.

Decoder Models: also called autoregressive models. Have access to words that come before and try predicting the next word (Causal Language Modelling). Useful for text generation, code-completion etc. gpt2, gpt-neo-125M

Encoder-Decoder Models: Sequence to sequence Models. Encoder reads the full input and decoder generates output sequentially based on the encoded input. Useful for summarization, text-translation. Typically the input is corrupted and the job of the model is to predict the correct input. Best suited for tasks that involve generating an output text given an input text. Encoder encodes the understanding of the whole text and decoder tries predicting the next one.

What makes these model effective? Attention. The process of identifying most relevant words to make a prediction.